# Section 1. Install and Import Dependencies

### Note: When running on Colab, make sure to set Hardware Accelarator to GPU

### 1.1 System configuration Check

In [ ]:
#check env for local run
import sys
sys.executable

In [ ]:
!nvidia-smi # for GPU specification

In [ ]:
!nvcc --version #to check CUDA version

In [ ]:
!cat /etc/os-release

In [ ]:
import torch

use_cuda = torch.cuda.is_available()
use_cuda

### 1.2 Pytorch installation

In [ ]:
if use_cuda:
    print('GPU found, proceeding with CUDA installation...')
    !pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
else:
    print('GPU is not found, proceeding with CPU version installation...')
    !pip install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

### all imports 

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
from datetime import datetime
import shutil
import zipfile
import pathlib
import random
import string
from os import listdir
from os.path import isfile, join

### 1.3 Clone ultralytics repo and install the requirements

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 && pip install -r requirements.txt

### 1.4 Define various paths and create folders

In [ ]:
paths = {
    'DATA_PATH'  : os.path.join('data'),
    'IMAGES_PATH': os.path.join('data', 'images'),
    'LABELS_PATH': os.path.join('data', 'labels'),
    'VIDEOS_PATH': os.path.join('data', 'videos'),
    'INFER_PATH' : os.path.join('data', 'inference')
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
            
        if os.name == 'nt':
            !mkdir {path}

# Section 2. Data Preparation (Local run only)

### Download required images from intenet and keep them in data/images folder

### 2.1 Rename images in serial order for easy reference. It will be like image001.jpg, image002.png etc..,

In [ ]:
def rename_files(path, file_index=1, prefix=None):
    
    # prefix: This is the common name part of the file name (program will autogenerate 5 letter code, if nothing is passed)
    # file_index: files will be renamed img001.jpg etc.., if 100, files will be renamed img100.jpg etc..,
    
    if prefix is None:
        prefix = ''.join(random.choices(string.ascii_uppercase, k=5))   

    for rootdir, subdir, files in os.walk(path):
        for file in files:
            file_extension = pathlib.Path(file).suffix.strip()
            old_file = os.path.join(path, file)
            new_file = os.path.join(path, prefix + f"{file_index:03}" + file_extension)
            os.rename(old_file, new_file)
            file_index += 1
            
    print('Files renamed as per "{}..." pattern.'.format(prefix)) 

In [ ]:
#rename the files from images folder
rename_files(paths['IMAGES_PATH'])

### 2.2 Use labelimg to do annotations in YOLO format. Image direcotry should be data/images and label directory should be data/labels 

### 2.3 Manually move files from images folder to images/train, images/validation so that folder contains a fair representation of all classes including background images. 

In [ ]:
paths.update ({
    'IMAGES_TRAIN': os.path.join('data', 'images', 'train'),
    'IMAGES_VAL': os.path.join('data', 'images', 'validation'),
    'LABELS_TRAIN': os.path.join('data', 'labels', 'train'),
    'LABELS_VAL': os.path.join('data', 'labels', 'validation'),
})

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
            
        if os.name == 'nt':
            !mkdir {path}

### 2.4 Then run the below script to move the respective label files to labels/train, labels/validation

In [ ]:
from os import listdir
from os.path import isfile, join

num_train = num_val = num_train_bg = num_val_bg = 0

train_files = [f for f in listdir(paths['IMAGES_TRAIN']) if isfile(join(paths['IMAGES_TRAIN'], f))]
val_files = [f for f in listdir(paths['IMAGES_VAL']) if isfile(join(paths['IMAGES_VAL'], f))]

for file in train_files:
    fname, fext = file.split('.')
    try:
        num_train += 1
        shutil.move(os.path.join(paths['LABELS_PATH'], fname + ".txt"), paths['LABELS_TRAIN'])         
    except Exception as e:
        num_train_bg += 1
        
for file in val_files:
    fname, fext = file.split('.')
    try:
        num_val += 1
        shutil.move(os.path.join(paths['LABELS_PATH'], fname + ".txt"), paths['LABELS_VAL'])         
    except Exception as e:
        num_val_bg += 1        
        
        
print('Train images: {}; Validation images: {}; Train Background images: {}; Validation Background images: {}'
                                  .format(num_train, num_val, num_train_bg, num_val_bg))
    

# Section 3. Data setup (for Colab only) 

### use one of the below options to get the data.

### 3.1 zip the local data folder, manually upload to colab and unzip using the below command

In [ ]:
# command to zip the local data folder
!tar -czf {'data_archive.tar.gz'} {paths['DATA_PATH']}
print('Archiving data file is now complete.')

In [ ]:
# command to unzip the folder from google colab
!tar -zxvf {'data_archive.tar.gz'}

### 3.2 Keep the data folder contents to your google drive and run the below command to copy to colab workspace

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change location as per your GD structure
!cp -r 'drive/MyDrive/Colab Notebooks/Git Projects Data/Wildlife Inference/data' .

### 3.3 Run the below commands to verify the number of images in each of the folders (local vs colab) are same

In [ ]:
!ls data/labels

In [ ]:
!ls data/images/train | wc -l
!ls data/images/validation | wc -l

In [ ]:
!ls data/labels/train | wc -l 
!ls data/labels/validation | wc -l

# Section 4. Load Out of box Model from torch hub

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # or yolov5m, yolov5l, yolov5x, custom

In [ ]:
model

## 4.1 Image Detection with Out of box model

In [ ]:
img = 'https://images.news18.com/ibnlive/uploads/2021/10/animal-day-16332888954x3.jpg'

In [ ]:
# img = 'https://ultralytics.com/images/zidane.jpg'

In [ ]:
results = model(img)
results.print()

In [ ]:
%matplotlib inline 
plt.rcParams["figure.figsize"] = (20,8)
plt.imshow(np.squeeze(results.render()))
plt.savefig(os.path.join(paths['INFER_PATH'], 'img_001' + '.png'))
plt.show()

In [ ]:
results.render()

# Section 5. Prepare custom images (For webcam use only)

In [ ]:
import uuid   # Unique identifier
import os
import time

In [ ]:
IMAGES_PATH = os.path.join('data', 'images') #/data/images
labels = ['awake', 'drowsy']
number_imgs = 5

In [ ]:
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    
    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam feed
        ret, frame = cap.read()
        
        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        # Writes out image to file 
        cv2.imwrite(imgname, frame)
        
        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(os.path.join(IMAGES_PATH, labels[0]+'.'+str(uuid.uuid1())+'.jpg'))

In [ ]:
for label in labels:
    print('Collecting images for {}'.format(label))
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        print(imgname)   

In [ ]:
!git clone https://github.com/tzutalin/labelImg

In [ ]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

# Section 6. Custom Training (after images, labels folders are updated)

### 6.1 Keep the .yaml file in GD as per the dataset and run the below command to copy it to yolov5 folder

In [ ]:
!cp 'drive/MyDrive/Colab Notebooks/Git Projects Data/Wildlife Inference/wildlife.yaml' 'yolov5'

### 6.2 If using past weights, keep it in GD and run the below command to copy them to yolov5 folder

In [ ]:
!cp -r 'drive/MyDrive/Colab Notebooks/Git Projects Data/Wildlife Inference/weights' 'yolov5'

### 6.3 Run the below cell to start the training process (change --weights param as needed to point local file)

In [ ]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 300 --data wildlife.yaml --weights yolov5s.pt --workers 2

### 6.4 zip the weights file on Colab space to manually export it to local machine

In [ ]:
# zip experminet folder and manually export to local machine
!tar -czf exp_small.tar.gz {'yolov5/runs/train/exp'}

In [ ]:
!ls -lh

# Section 7. Custom Model Inference

### 7.1 Load Custom model

In [ ]:
if use_cuda:
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/best.pt', force_reload=True)
else:
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='exp/weights/best.pt', force_reload=True)

## 7.2 Image inference

In [ ]:
img = 'https://images.news18.com/ibnlive/uploads/2021/10/animal-day-16332888954x3.jpg'

In [ ]:
results = model(img)
results.print()

In [ ]:
%matplotlib inline 
plt.rcParams["figure.figsize"] = (20,8)
plt.imshow(np.squeeze(results.render()))
plt.savefig(os.path.join(paths['INFER_PATH'], 'img_001' + '.png'))
plt.show()

## 7.3 Recorded Video Inference

In [ ]:
video_name = 'Wildlife-002.mp4' 
video_input = os.path.join(paths['VIDEOS_PATH'], video_name)

import tkinter as tk

root = tk.Tk()

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

screen_width, screen_height

In [ ]:
from datetime import datetime
import time

start_time = datetime.now()

print('Inference started at: {}'.format(start_time.strftime("%I:%M:%S %p")))

cap = cv2.VideoCapture(video_input) # 0 for webcam
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
fps = cap.get(cv2.CAP_PROP_FPS)  # 15.0, set the video fps
out = cv2.VideoWriter(os.path.join(paths['INFER_PATH'], video_name), fourcc, fps, (screen_width, screen_height))
time.sleep(0.5) #delay to allow camera to load the feed

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break    
    
    #make detections
    results = model(frame)
    op = np.squeeze(results.render())
    out.write(cv2.resize(op, (screen_width, screen_height))) # (640, 480), (1280, 720), (screen_width, screen_height)    

    #temporarily, comment the below cv2.imshow line for colab run. To be fixed
    cv2.imshow('YOLOv5 Inference - Recorded video', op)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

end_time = datetime.now()

print('Inference ended at: {}'.format(end_time.strftime("%I:%M:%S %p")))
print('Time taken for inference: {}'.format(end_time - start_time))                  
